In [1]:
import torch
import pandas as pd
import numpy as np
import pytorch_transformers

folder = 'd:\git\project1\generated'

In [2]:
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

100%|███████████████████████████████████████████████████████████████████████| 231508/231508 [00:01<00:00, 150095.15B/s]


In [18]:
df_small = pd.read_csv(f'{folder}/small_news.csv')
del df_small['Unnamed: 0']
del df_small['Unnamed: 0.1']
df_small

,Date,Headline
0,2018-04-26T11:31:02+00:00,Square Books Small Profit for First Quarter of...
1,2018-05-03T15:30:59+00:00,Daily Volatility Decline? Bitcoin Has Seen $1K...
2,2018-05-03T15:30:59+00:00,Sell In May and Go Away? Not for Bitcoin Bulls
3,2018-05-02T21:10:20+00:00,Reddit to Relaunch Bitcoin Payments (And Add M...
4,2018-05-03T18:00:39+00:00,Bitcoin Futures Trading Questioned By Chinese ...
...,...,...
16601,"\r\r\n Updated April 9, 2013 2:48 p.m. ET\r\r\n",Tech Entrepreneurs Invest in Trading Platform ...
16602,"\r\r\n April 4, 2013\r\r\n",Eine Kleine Trauermusik
16603,"\r\r\n March 29, 2013 11:34 a.m. ET\r\r\n",Cyprus and Cybermoney
16604,"\r\r\n Updated March 21, 2013 10:02 p.m. ET\r...",Web Money Gets Laundering Rule


In [30]:
df_small.sort_values(by='date')

,date,time,title
140322,1546214400,15583,IMPORTANT INFORMATION TO KEEP IN MIND WHEN CHO...
42421,1546214400,69176,Humanity 'Sleepwalking Towards the Edge of a C...
302926,1546214400,77484,France's Macron pledges more reform medicine i...
396456,1546214400,9608,India university official urges students to ki...
261637,1546214400,79763,Republican candidate kept donations despite re...
...,...,...,...
2107,1617926400,73591,Are You Confused by Scientific Jargon? So Are ...
422120,1617926400,50931,Ukraine fears attack as Putin masses tanks clo...
80331,1617926400,62165,Judge drops corruption charges against Portuga...
103957,1617926400,50698,You are about to embark on a journey that can ...


In [17]:
import re
def is_btc(cell):
    return bool(not re.search(r'[btc|bitcoin]', cell))

df_small['bitcoin'] = df_small['title'].apply(lambda x: (bool)(re.search(r'\bbtc\b|\bbitcoin\b|\busd\b|\bdollar\b', x)))
df_btc = df_small[~(df_small['bitcoin'] == False)]
df_btc

,date,time,title,bitcoin
146,1565049600,47445,China sets its yuan midpoint at stronger than ...,True
207,1556928000,73414,Chinese banks quietly lower limit on US dollar...,True
1316,1613952000,21308,Tesla has made about $1 billion in profit on i...,True
2751,1604102400,79620,$100 dollar oil makes since if you follow basi...,True
2754,1584748800,49220,Ottawa prepares multibillion-dollar bailout of...,True
...,...,...,...,...
541287,1613606400,14108,A mysterious entity bought 20 thousand bitcoin...,True
548815,1591228800,45148,Carnival slapped with 2nd multimillion-dollar ...,True
549434,1578268800,71007,Alleged mastermind behind multibillion dollar ...,True
549617,1569196800,54880,The AK 47 costs 130 dollar at a donkey fair in...,True


In [13]:
df_btc.loc[1316, 'title']

'Tesla has made about $1 billion in profit on its bitcoin investment, analyst estimates'

In [8]:
df_btc = pd.read_csv(f'{folder}/bts_price_changes.csv')
df_btc.timestamp

0       1410912000
1       1410998400
2       1411084800
3       1411171200
4       1411257600
           ...    
2395    1617926400
2396    1618012800
2397    1618099200
2398    1618185600
2399    1618272000
Name: timestamp, Length: 2400, dtype: int64

In [29]:
df_union = pd.DataFrame()
df_union = df_small.loc[df_small['date'] == 1411171200]
df_union

,date,time,title


In [ ]:
tokenized = df_small['title'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

In [ ]:
def token_lenght(cell, max):
    if (len(cell) <= max):
        return cell

max = 30
tokens = pd.DataFrame()
tokens['raw'] = tokenized.apply(lambda x: x if len(x) > max)
tokens = tokens.dropna()
#tokens = tokenized.apply(token_lenght, max=30)

tokens

In [ ]:
max_len = 0
for i in tokens['raw']:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

In [ ]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokens['raw']])

In [ ]:
df_padded = pd.DataFrame(padded)
df_padded

In [ ]:
df_union = df_small.merge(df_padded, left_index=True, right_index=True, how='inner')
df_union

In [ ]:
df_union.to_csv(f'{folder}/date_time_tokens_array_short.csv')

In [ ]:
tokens_padded = df_union.iloc[:, 4:]
tokens_padded.to_csv(f'{folder}/tokens_padded.csv')
tokens_padded

In [ ]:
date_time_title_short = df_union.iloc[:, 1:4]
date_time_title_short.to_csv(f'{folder}/date_time_title_short.csv')